In [1]:
# Thank you to https://stmorse.github.io/journal/spotify-api.html for getting started
import pandas as pd # csv and dataframes
import requests # Spotify API
import config_secrets # Retrieve my secrets from this file

# Secrets from developer.spotify.com account
# You would have to change this
CLIENT_ID = config_secrets.CLIENT_ID
CLIENT_SECRET = config_secrets.CLIENT_SECRET

# How to get a token
AUTH_URL = 'https://accounts.spotify.com/api/token'

# Where to send spotify GET requests
BASE_URL = 'https://api.spotify.com/v1/'

In [2]:
# POST to get our access token
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET
})

auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']

# Include this in every GET request to make sure we have access
headers = {'Authorization': f"Bearer {access_token}"}

In [3]:
### CHANGE THIS ###############
playlist_name = "This Is Raffi"
playlist_id = '37i9dQZF1DZ06evO4nGfER'
###############################
r_playlist = requests.get(BASE_URL + f"playlists/{playlist_id}/tracks", headers=headers)
df_playlist = pd.json_normalize(r_playlist.json()['items'])
df_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 40 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   added_at                            46 non-null     object
 1   is_local                            46 non-null     bool  
 2   primary_color                       0 non-null      object
 3   added_by.external_urls.spotify      46 non-null     object
 4   added_by.href                       46 non-null     object
 5   added_by.id                         46 non-null     object
 6   added_by.type                       46 non-null     object
 7   added_by.uri                        46 non-null     object
 8   track.album.album_type              46 non-null     object
 9   track.album.artists                 46 non-null     object
 10  track.album.available_markets       46 non-null     object
 11  track.album.external_urls.spotify   46 non-null     object
 

In [4]:
# Get audio features described at the link below (up to 100 songs at a time)
# https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features
r_features = requests.get(BASE_URL + 'audio-features?ids=' + ','.join(df_playlist['track.id']), headers=headers)
df_features = pd.json_normalize(r_features.json()['audio_features'])
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      46 non-null     float64
 1   energy            46 non-null     float64
 2   key               46 non-null     int64  
 3   loudness          46 non-null     float64
 4   mode              46 non-null     int64  
 5   speechiness       46 non-null     float64
 6   acousticness      46 non-null     float64
 7   instrumentalness  46 non-null     float64
 8   liveness          46 non-null     float64
 9   valence           46 non-null     float64
 10  tempo             46 non-null     float64
 11  type              46 non-null     object 
 12  id                46 non-null     object 
 13  uri               46 non-null     object 
 14  track_href        46 non-null     object 
 15  analysis_url      46 non-null     object 
 16  duration_ms       46 non-null     int64  
 17 

In [5]:
# Join the original table with the data we got from the Spotify API
data = df_playlist.join(df_features.set_index('id'), on='track.id', how='inner')
data.info() # Look at those columns

<class 'pandas.core.frame.DataFrame'>
Index: 46 entries, 0 to 45
Data columns (total 57 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   added_at                            46 non-null     object 
 1   is_local                            46 non-null     bool   
 2   primary_color                       0 non-null      object 
 3   added_by.external_urls.spotify      46 non-null     object 
 4   added_by.href                       46 non-null     object 
 5   added_by.id                         46 non-null     object 
 6   added_by.type                       46 non-null     object 
 7   added_by.uri                        46 non-null     object 
 8   track.album.album_type              46 non-null     object 
 9   track.album.artists                 46 non-null     object 
 10  track.album.available_markets       46 non-null     object 
 11  track.album.external_urls.spotify   46 non-null     

In [6]:
data.head() # Look at the first few rows of data

,added_at,is_local,primary_color,added_by.external_urls.spotify,added_by.href,added_by.id,added_by.type,added_by.uri,track.album.album_type,track.album.artists,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,2023-09-10T11:18:03Z,False,None,https://open.spotify.com/user/,https://api.spotify.com/v1/users/,,user,spotify:user:,album,[{'external_urls': {'spotify': 'https://open.s...,...,0.000000,0.3460,0.671,185.302,audio_features,spotify:track:0e1mUAXE8jSG9ExAS4AWU7,https://api.spotify.com/v1/tracks/0e1mUAXE8jSG...,https://api.spotify.com/v1/audio-analysis/0e1m...,65053,4
1,2023-09-10T11:18:03Z,False,None,https://open.spotify.com/user/,https://api.spotify.com/v1/users/,,user,spotify:user:,album,[{'external_urls': {'spotify': 'https://open.s...,...,0.000002,0.4520,0.571,137.520,audio_features,spotify:track:7zQFM7bHcyMh4QQFdBkjrE,https://api.spotify.com/v1/tracks/7zQFM7bHcyMh...,https://api.spotify.com/v1/audio-analysis/7zQF...,160827,4
2,2023-09-10T11:18:03Z,False,None,https://open.spotify.com/user/,https://api.spotify.com/v1/users/,,user,spotify:user:,album,[{'external_urls': {'spotify': 'https://open.s...,...,0.043100,0.1670,0.484,97.764,audio_features,spotify:track:2Qu65QrO1DiWVJIteOD9ri,https://api.spotify.com/v1/tracks/2Qu65QrO1DiW...,https://api.spotify.com/v1/audio-analysis/2Qu6...,71160,4
3,2023-09-10T11:18:03Z,False,None,https://open.spotify.com/user/,https://api.spotify.com/v1/users/,,user,spotify:user:,album,[{'external_urls': {'spotify': 'https://open.s...,...,0.000000,0.1020,0.869,204.418,audio_features,spotify:track:6jTZGXH1cOQM3jhh07ZE1y,https://api.spotify.com/v1/tracks/6jTZGXH1cOQM...,https://api.spotify.com/v1/audio-analysis/6jTZ...,128213,4
4,2023-09-10T11:18:03Z,False,None,https://open.spotify.com/user/,https://api.spotify.com/v1/users/,,user,spotify:user:,album,[{'external_urls': {'spotify': 'https://open.s...,...,0.000000,0.0875,0.792,128.483,audio_features,spotify:track:1ePn7ToIaezbA4tlmj4Kgh,https://api.spotify.com/v1/tracks/1ePn7ToIaezb...,https://api.spotify.com/v1/audio-analysis/1ePn...,71040,4


In [7]:
# Fix track.artists so we only see the first artist
data['track.artists'] = [data['track.artists'][x][0]['name'] for x in range(len(data['track.artists']))]

In [8]:
data.loc[0]

added_at                                                           2023-09-10T11:18:03Z
is_local                                                                          False
primary_color                                                                      None
added_by.external_urls.spotify                           https://open.spotify.com/user/
added_by.href                                         https://api.spotify.com/v1/users/
added_by.id                                                                            
added_by.type                                                                      user
added_by.uri                                                              spotify:user:
track.album.album_type                                                            album
track.album.artists                   [{'external_urls': {'spotify': 'https://open.s...
track.album.available_markets         [AR, AU, AT, BE, BO, BR, BG, CL, CO, CR, CY, C...
track.album.external_urls.spotif

In [9]:
data.to_csv(f"data/{playlist_name}.csv")